# Preparing a terra.bio sample table from GEO/SRA
TODO

In [1]:
# bioproject <- "PRJNA668299" # Mellor Lab - Spt4
bioproject <- "PRJNA669852"# Churchman Lab - dozens of regulatory factors
genomeName <- "sacCer3"
genome_fasta <- "https://hgdownload.soe.ucsc.edu/goldenPath/sacCer3/bigZips/sacCer3.fa.gz"
genome_local_name <- "sacCer3.fa"

In [2]:
# Load needed packages, installing if necessary
required_packages <- c("AnVIL", "xml2", "rentrez", "glue", "kableExtra",
                      "tidyverse")
install_and_load <- function(packages) {
    if (length(setdiff(packages, rownames(installed.packages()))) > 0) {
        BiocManager::install(setdiff(packages, rownames(installed.packages())))
    }
    lapply(packages, library,  character.only = TRUE)
    NULL
}
x <- install_and_load(required_packages)

Lade n"otiges Paket: dplyr


Attache Paket: 'dplyr'


Die folgenden Objekte sind maskiert von 'package:stats':

    filter, lag


Die folgenden Objekte sind maskiert von 'package:base':

    intersect, setdiff, setequal, union



Attache Paket: 'glue'


Das folgende Objekt ist maskiert 'package:dplyr':

    collapse



Attache Paket: 'kableExtra'


Das folgende Objekt ist maskiert 'package:dplyr':

    group_rows


-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.5     v stringr 1.4.0
v tidyr   1.1.4     v forcats 0.5.1
v readr   2.0.2     

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x glue::collapse()         masks dplyr::collapse()
x dplyr::filter()          masks stats::filter()
x purrr::flatten()         masks AnVIL::flatten()
x kableExtra::group_rows() masks dplyr::group_rows()
x dplyr::lag()             masks stats::lag()



In [3]:
# Define functions
# NOTE: use rentrez to avoid using the SRAdb secondary source.
# filetype in "fastq", "run"
create_sample_grid <- function(bioproject,
            semantic_name = "fastq", org = "GCP", retmax = NULL) {

        bpid <- entrez_search(db = "bioproject", term = bioproject)
    if (bpid$count == 0) {
        stop(glue::glue("Bioproject <{bioproject}> not found"))
    }

    sra_ids <- entrez_link(dbfrom = "bioproject", id = bpid$ids, db = "sra")$links$bioproject_sra
    x <- read_xml(entrez_fetch(db = "sra", id = sra_ids, rettype = "xml", retmax = retmax))
    
    runs <- xml_find_all(x, '/EXPERIMENT_PACKAGE_SET/EXPERIMENT_PACKAGE/RUN_SET/RUN')
    run_id <- xml_attr(runs, "accession")
    expref <- xml_find_all(runs, "EXPERIMENT_REF")
    experiment_id <- xml_attr(expref, "accession")
    biosample_id <- xml_attr(expref, "refname")
    total_reads <- xml_attr(runs, "total_spots")
    total_bases <- xml_attr(runs, "total_bases")
    size <- xml_attr(runs, "size")
    # Only reporting first member in pool
    member <- xml_find_first(runs, "Pool/Member")
    sample_title <- xml_attr(member, "sample_title")
    sra_sample_id <- xml_attr(member, "accession")
    sra_File <- xml_find_first(runs, glue("./SRAFiles/SRAFile[@semantic_name=\"{semantic_name}\"]"))
    sra_File_alt <- xml_find_first(sra_File, glue("./Alternatives[@org=\"{org}\"]"))
    filename <- xml_attr(sra_File, "filename")
    sra_url <- xml_attr(sra_File_alt, "url")
    tibble(bioproject, experiment_id, biosample_id, sra_sample_id, run_id, sample_title, total_reads, total_bases, size, filename, sra_url)
}


In [4]:
# TODO unwrap create_sample_grid
bpid <- entrez_search(db = "bioproject", term = bioproject)
bp_summary <- entrez_summary(db = "bioproject", id = bpid$ids)
sapply(c("project_acc",
    "project_name",
    "submitter_organization",
    "registration_date"), 
       function(u) data.frame(attribute = u, value = bp_summary[[u]]), USE.NAMES = FALSE) %>% 
 t %>% kable(format = "pipe", caption = "Bioproject Summary Attributes")



Table: Bioproject Summary Attributes

|attribute              |value                                                                                 |
|:----------------------|:-------------------------------------------------------------------------------------|
|project_acc            |PRJNA669852                                                                           |
|project_name           |Dynamics of transcription elongation are finely-tuned by dozens of regulatory factors |
|submitter_organization |Stirling Churchman, Genetics, Harvard Medical School                                  |
|registration_date      |2020/10/19 00:00                                                                      |

In [8]:
result <- create_sample_grid(bioproject)

# TODO Clean up sample_id's if we are going to allow multiple assays
# Infer strain and assay type from sample_title
result %>% 
    separate(sample_title, into = c("sample_id", "assay"), sep = "_", remove = FALSE) %>%
    separate(sample_id, into = "strain", sep = "-", extra = "drop", remove = FALSE) %>%
    filter(assay == "NETseq") %>%
    mutate(genome = genomeName) %>%
    relocate(sample_id) %>%
    arrange(sample_title) %>%
    dplyr::rename(inputFastQ = sra_url, "entity:sample_id" = sample_id) -> sample


sample %>% avtable_import


Warning message in if (!nzchar(namespace)) {:
"Bedingung hat L"ange > 1 und nur das erste Element wird benutzt"


ERROR: Error in avtable_import(.): .is_scalar_character(namespace) ist nicht TRUE


In [17]:
#sample
str(import_result)
dim(sample)
#print(glue("Table {import_result}, {nrow(sample)} rows Created."))


ERROR: Error in h(simpleError(msg, call)): Fehler bei der Auswertung des Argumentes 'object' bei der Methodenauswahl f"ur Funktion 'str': Objekt 'import_result' nicht gefunden


In [ ]:
# Copy genome fasta to local bucket
work_dir <- tempdir()
genome_local_fa <- glue("{work_dir}/{genome_local_name}")
genome_local_gz <- glue("{genome_local_fa}.gz")
download.file(genome_fasta, genome_local_gz)
# TODO Assumes we are in gzip format. For simplicity, save uncompressed for now ... 
system(glue("gunzip -f {genome_local_gz}"))
gs_uri <- glue("{avbucket()}/{genome_local_name}")
gsutil_cp(genome_local_fa, gs_uri)

In [ ]:
work_dir

In [ ]:
# genome table
tibble(`entity:genome_id` = genomeName, refFasta = glue("\"{gs_uri}\"")) %>% avtable_import